In [1]:
from HodaDatasetReader import read_hoda_dataset


In [2]:
X_train, Y_train = read_hoda_dataset(dataset_path='./DigitDB/Train 60000.cdb',
                                images_height=32,
                                images_width=32,
                                one_hot=False,
                                reshape=True)
X_test, Y_test = read_hoda_dataset(dataset_path='./DigitDB/Test 20000.cdb',
                              images_height=32,
                              images_width=32,
                              one_hot=False,
                              reshape=True)
X_val, Y_val = read_hoda_dataset('./DigitDB/RemainingSamples.cdb',
                                             images_height=32,
                                             images_width=32,
                                             one_hot=False,
                                             reshape=True)

In [3]:
import tensorflow as tf 


In [4]:
# foe simplicity, we will scale the pixel intensities (range : 0-1)
X_train,X_test,X_val=X_train / 255.,X_test/ 255.,X_val/ 255.

#### sequential model

In [5]:
# make the results reproducible
tf.random.set_seed(42)
model= tf.keras.Sequential([tf.keras.layers.Dense(300,activation='relu'),tf.keras.layers.Dense(300,activation='relu'),tf.keras.layers.Dense(300,activation='softmax')])


Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-15 21:20:26.725521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-15 21:20:26.725784: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [7]:
model.fit(X_train,Y_train,epochs=40,validation_data=(X_val,Y_val))

Epoch 1/40


2023-02-15 21:20:26.940284: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


   1/1875 [..............................] - ETA: 15:22 - loss: 5.7037 - accuracy: 0.0000e+00

2023-02-15 21:20:27.269589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1870/1875 [============================>.] - ETA: 0s - loss: 2.9747 - accuracy: 0.1017

2023-02-15 21:20:46.354140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 22s 11ms/step - loss: 2.9729 - accuracy: 0.1018 - val_loss: 2.3109 - val_accuracy: 0.1042
Epoch 2/40
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3091 - accuracy: 0.1037 - val_loss: 2.3091 - val_accuracy: 0.0860
Epoch 3/40
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3072 - accuracy: 0.1063 - val_loss: 2.3089 - val_accuracy: 0.0860
Epoch 4/40
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3065 - accuracy: 0.1061 - val_loss: 2.3108 - val_accuracy: 0.0860
Epoch 5/40
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3050 - accuracy: 0.1062 - val_loss: 2.3051 - val_accuracy: 0.1013
Epoch 6/40
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3034 - accuracy: 0.1095 - val_loss: 2.3027 - val_accuracy: 0.1008
Epoch 7/40
1875/1875 [==============================] - 10s 5ms/step - loss: 2.3023 - accuracy: 0.1112 - val_loss: 2.3033 - val_accuracy: 0.